In [ ]:
import numpy as np
import random

In [ ]:
population_size=100
dimension_ackley=5
dimension_sphere=10
dimension_max=10
shift=[random.uniform(-1, 1) for _ in range(dimension_ackley)]

In [ ]:
frequence_translate=0
threshold=0
maxgen=400
cp=np.full(dimension_max, 0.5)
sf=1.25


In [ ]:
def ackley_function(x,shift):
    x=x[:dimension_ackley]
    n = len(x)
    sum_sq_term = -0.2 * np.sqrt((1/n) * np.sum((x - shift) ** 2))
    cos_term = np.sum(np.cos(2 * np.pi * (x - shift)))
    return -20 * np.exp(sum_sq_term) - np.exp((1/n) * cos_term) + 20 + np.exp(1)

def sphere_function(x):
    x=x[:dimension_sphere]
    return np.sum(x**2)

In [ ]:
def initialize_population(population_size, dimension_max):
    return np.random.uniform(-5.12, 5.12, size=(population_size, dimension_max))

In [ ]:
def cost_ackley(population):
    result = []
    for x in population:
      result.append(ackley_function(x,shift));
    return np.array(result)
def cost_sphere(population):
    result = []
    for x in population:
        result.append(sphere_function(x));
    return np.array(result)

In [ ]:
def rank_ackley(population):
    return np.argsort(np.argsort(cost_ackley(population)))
def rank_sphere(population):
    return np.argsort(np.argsort(cost_sphere(population)))

In [ ]:
def evalution(population):
  rank_1=rank_ackley(population)
  rank_2=rank_sphere(population)
  skill_factor=[];
  for i in range(population_size):
        if rank_1[i] < rank_2[i]:
            skill_factor.append(0)
        else:
            skill_factor.append(1)
  return np.array(skill_factor)

In [ ]:
def crossover(parent1, parent2):
    cf=random.random()
    child1 =0.5*( parent1 * (1+cf) + parent2 * (1 - cf))
    child2 =0.5*( parent2 * (1+cf) + parent1 * (1 - cf))
    return child1, child2

In [ ]:
def mutation(individual, mutation_rate, mutation_scale):
    mutated_individual = individual.copy()
    for i in range(len(mutated_individual)):
        if (np.random.rand() < mutation_rate ):
            mutated_individual[i] += np.random.normal(scale=mutation_scale)
    return mutated_individual


In [ ]:
def get_m_k_sphere(population):
  sphere_values = cost_sphere(population)
  sorted_indices = np.argsort(sphere_values)
  num_elements = int(population_size * 0.4)
  selected_indices = sorted_indices[:num_elements]
  selected_points = population[selected_indices]
  return np.sum(selected_points, axis=0)/num_elements
def get_m_k_ackley(population):
  ackley_values = cost_ackley(population)
  sorted_indices = np.argsort(ackley_values)
  num_elements = int(population_size * 0.4)
  selected_indices = sorted_indices[:num_elements]
  selected_points = population[selected_indices]
  return np.sum(selected_points, axis=0)/num_elements

In [ ]:
def translate(gen, maxgen, population,frequency,threshold,skill_factor,d_k_ackley,d_k_sphere):
  NP=population.shape[0]
  population_=[]
  if(gen>threshold):
    if(gen%frequency==0):
      alpha=(gen/maxgen)*(gen/maxgen)
      d_k_ackley=sf*alpha*(cp-get_m_k_ackley(population))
      d_k_sphere=sf*alpha*(cp-get_m_k_sphere(population))
    for i in range(NP):
      if(skill_factor[i]==0):
        p_=population[i]+d_k_ackley
      else:
        p_=population[i]+d_k_sphere
      population_.append(p_)
  else:
    population_=population
  return population_,d_k_ackley,d_k_sphere

In [ ]:
def shuffle(parent1_,parent2_,population_,skill_factor,skill_factor_1,skill_factor_2):
  D1=len(parent1_)
  D2=len(parent2_)
  Dmax=max(D1,D2)
  l1=[i for i in range(0, Dmax)]
  l2=[i for i in range(0, Dmax)]
  if D1<D2:
    random.shuffle(l1)
    p_=population_[skill_factor.index(skill_factor_2)]
    for i in range(D1):
      p_[l1[i]]=parent1_[i]
    return p_,parent2_,l1,l2
  elif D1>D2:
    random.shuffle(l2)
    p_=population_[skill_factor.index(skill_factor_1)]
    for i in range(D2):
      p_[l2[i]]=parent2_[i]
    return parent1_,p_,l1,l2
  else:
    return parent1_,parent2_,l1,l2

In [ ]:
def re_transfer(o1__,o2__,l1,l2,len1,len2):
  if len1>len2:
    o2_=[None]*len2
    for i in range(len(l1)):
      o2_[l2[i]]=o2__[i]
    return o1__,o2_
  elif len1<len2:
    o1_=[None]*len1
    for i in range(len(l2)):
      o1_[l1[i]]=o1__[i]
    return o1_,o2__
  else:
    return o1__,o2__

In [ ]:
def assortative_mating(p1,p2, skill_factor1,skill_factor2):
    rmp=0.3
    rnd = np.random.rand(1)[0]
    if skill_factor1== skill_factor2 or rnd < rmp:
      o1,o2=crossover(p1,p2);
      random_skill_factor = np.random.rand(1)[0]
      if random_skill_factor < 0.5:
        return o1,o2,skill_factor1,skill_factor1
      else:
        return o1,o2,skill_factor2,skill_factor2
    else:
        o1= p1.copy()
        o2 = p2.copy()
        o1=mutation(o1,0.1,0.1);
        o2=mutation(o2,0.1,0.1);
        return o1,o2,skill_factor1,skill_factor2

In [ ]:
def update_scalar_fitness(population, s):
    ackley_pop = population[np.where(s == 0)[0]]
    rank1 = rank_ackley(ackley_pop)
    scalar_fitness_ackley = [1/(i+1) for i in rank1]

    sphere_pop = population[np.where(s == 1)[0]]
    rank2 = rank_sphere(sphere_pop)
    scalar_fitness_sphere = [1/(i+1) for i in rank2]
    scalar_fitness = np.random.rand(population.shape[0])
    scalar_fitness[np.where(s == 0)[0]] = scalar_fitness_ackley
    scalar_fitness[np.where(s == 1)[0]] = scalar_fitness_sphere
    resize=len(scalar_fitness)-population_size
    filter_indices = np.argsort(scalar_fitness)[:resize]
    fittest_pop = np.delete(population, filter_indices, axis=0)
    fittest_skill_factor = np.delete(s, filter_indices)
    return fittest_pop, fittest_skill_factor


In [ ]:
def GMFEAs(NP):
  population_=[]
  d_k_ackley=0;
  d_k_sphere=0
  frequency=int(0.02*maxgen)
  threshold=int(0.1*maxgen)
  population=initialize_population(NP,dimension_max)
  skill_factor=evalution(population)
  for i in range(maxgen):
    population_,d_k_ackley,d_k_sphere=translate(i+1,maxgen,population,frequency,threshold,skill_factor,d_k_ackley,d_k_sphere)
    offspring=[]
    skill_factor_offspring=[]
    for j in range(int(NP/2)):
       random_indices = np.random.choice(np.arange(len(population_)), size=2, replace=False)
       p1_,p2_=population_[random_indices[0]],population_[random_indices[1]]
       p1__,p2__,l1,l2=shuffle(p1_,p2_,population_,skill_factor,skill_factor[random_indices[0]],skill_factor[random_indices[1]])
       o1__,o2__,skill_factor1,skill_factor2=assortative_mating(p1__,p2__,skill_factor[random_indices[0]],skill_factor[random_indices[1]])
       o1_,o2_=re_transfer(o1__,o2__,l1,l2,len(p1_),len(p2_))
       if(skill_factor1==0):
        o1=o1_-d_k_ackley
       else:
        o1=o1_-d_k_sphere
       if(skill_factor2==0):
        o2=o2_-d_k_ackley
       else:
        o2=o2_-d_k_sphere
       offspring.append(o1)
       offspring.append(o2)
       skill_factor_offspring.append(skill_factor1)
       skill_factor_offspring.append(skill_factor2)
    intermediate_pop = np.vstack((population, offspring))
    intermediate_skill_factor = np.concatenate((skill_factor,skill_factor_offspring))
    population, skill_factor = update_scalar_fitness(intermediate_pop, intermediate_skill_factor)
  ackley_pop = population[np.where(skill_factor == 0)[0]]
  sphere_pop = population[np.where(skill_factor == 1)[0]]
  print("Ackley: ", cost_ackley(ackley_pop).min())
  print("Sphere: ", cost_sphere(sphere_pop).min())

In [ ]:
GMFEAs(population_size)

Ackley:  0.0020970746205191126
Sphere:  6.59712783601495e-06
